## Importar bibliotecas

In [2]:
import pymysql
import pandas as pd
import numpy as np

## Crear la base de datos

In [2]:
# Creamos una conección
conn = pymysql.connect(
    host='localhost',
    user='root',
    passwd='1234567890'
)

cur = conn.cursor()

# Se elimina la base de datos si exixte y se vuelve a crear
cur.execute("DROP DATABASE IF EXISTS streaming_platforms")
cur.execute("CREATE DATABASE streaming_platforms")

conn.close()

## Crear tablas

In [3]:
# Creamos una conección a la base de datos
conn = pymysql.connect(
    host='localhost',
    user='root',
    passwd='1234567890',
    database = 'streaming_platforms'
)

cur = conn.cursor()

In [4]:
query = """
CREATE TABLE Actor(
    IdActor INT NOT NULL,
    Name VARCHAR(110),
    PRIMARY KEY (IdActor)
);
"""
cur.execute(query)

0

In [5]:
query = """
CREATE TABLE Platform(
    IdPlatform INT NOT NULL,
    Platform VARCHAR(10),
    PRIMARY KEY (IdPlatform)
);
"""
cur.execute(query)

0

In [6]:
query = """
CREATE TABLE Genr(
    IdGenr INT NOT NULL AUTO_INCREMENT,
    Genr VARCHAR(30),
    PRIMARY KEY (IdGenr)
);
"""
cur.execute(query)

0

In [7]:
query = """
CREATE TABLE Types(
    IdType INT NOT NULL AUTO_INCREMENT,
    Type VARCHAR(15),
    PRIMARY KEY (IdType)
);
"""
cur.execute(query)

0

In [8]:
query = """
CREATE TABLE Movie(
    IdMovie INT NOT NULL,
    Title VARCHAR(150),
    IdType INT NOT NULL,
    IdPlatform INT NOT NULL,
    ReleaseYear INT(5),
    Duration INT(4),
    TimeUnit VARCHAR(10),
    PRIMARY KEY (IdMovie),
    FOREIGN KEY (IdType) REFERENCES TypeS(IdType),
    FOREIGN KEY (IdPlatform) REFERENCES Platform(IdPlatform)
);
"""
cur.execute(query)

0

In [9]:
query = """
CREATE TABLE Movie_Actor(
    IdMovie INT NOT NULL,
    IdActor INT NOT NULL,
    FOREIGN KEY (IdMovie) REFERENCES Movie(IdMovie),
    FOREIGN KEY (IdActor) REFERENCES Actor(IdActor)
);
"""
cur.execute(query)

0

In [10]:
query = """
CREATE TABLE Movie_Genr(
    IdMovie INT NOT NULL,
    IdGenr INT NOT NULL,
    FOREIGN KEY (IdMovie) REFERENCES Movie(IdMovie),
    FOREIGN KEY (IdGenr) REFERENCES Genr(IdGenr)
);
"""
cur.execute(query)

0

## Ingestar datos

In [11]:
Type = pd.read_csv('./Tablas/Type.csv')
Type

,IdType,Type
0,1,Movie
1,2,TV Show


In [12]:
for i in range(Type.shape[0]):
    query ="""
    INSERT INTO Types (IdType, Type)
    VALUES ({},'{}');
    """.format(Type.IdType.iloc[i],Type.Type.iloc[i])
    cur.execute(query)
conn.commit()

In [13]:
Platform = pd.read_csv('./Tablas/Platform.csv')
Platform

,IdPlatform,Platform
0,1,Amazon
1,2,Disney
2,3,Hulu
3,4,Netflix


In [14]:
for i in range(Platform.shape[0]):
    query ="""
    INSERT INTO Platform (IdPlatform, Platform)
    VALUES ({},'{}');
    """.format(Platform.IdPlatform.iloc[i],Platform.Platform.iloc[i])
    cur.execute(query)
conn.commit()

In [15]:
Actor = pd.read_csv('./Tablas/Actor.csv')
Actor.iloc[0,1] = 'Desconocido'
Actor.head(3)

,IdActor,Name
0,1,Desconocido
1,2,Richard Widmark
2,3,NaN


In [16]:
for i in range(Actor.shape[0]):
    query ="""
    INSERT INTO Actor (IdActor, Name)
    VALUES ({},"{}");
    """.format(Actor.IdActor.iloc[i],Actor.Name.iloc[i])
    cur.execute(query)
conn.commit()

In [17]:
Genr = pd.read_csv('./Tablas/Genr.csv')
Genr.head(3)

,IdGenr,Genr
0,1,Crime
1,2,Science Fiction
2,3,Crime TV Shows


In [18]:
for i in range(Genr.shape[0]):
    query ="""
    INSERT INTO Genr (IdGenr, Genr)
    VALUES ({},"{}");
    """.format(Genr.IdGenr.iloc[i],Genr.Genr.iloc[i])
    cur.execute(query)
conn.commit()

In [19]:
Movie = pd.read_csv('./Tablas/Movie.csv')
Movie.head(3)

,IdMovie,Title,IdType,IdPlatform,ReleaseYear,Duration,TimeUnit
0,1,The Grand Seduction,1,1,2014,113.0,min
1,2,Take Care Good Night,1,1,2018,110.0,min
2,3,Secrets of Deception,1,1,2017,74.0,min


In [20]:
Movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdMovie      22998 non-null  int64  
 1   Title        22998 non-null  object 
 2   IdType       22998 non-null  int64  
 3   IdPlatform   22998 non-null  int64  
 4   ReleaseYear  22998 non-null  int64  
 5   Duration     22817 non-null  float64
 6   TimeUnit     22998 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 1.2+ MB


In [21]:
Movie.Duration.fillna('NULL', inplace=True)

In [22]:
for i in range(Movie.shape[0]):
    if Movie.Duration.iloc[i] != 'NULL':
        duration = int(Movie.Duration.iloc[i])
    else:
        duration = 'NULL'
    
    query ="""
    INSERT INTO Movie (IdMovie, Title, IdType, IdPlatform, ReleaseYear, Duration, TimeUnit)
    VALUES ({},"{}",{},{},{},{},"{}");
    """.format(Movie.IdMovie.iloc[i], str(Movie.Title.iloc[i]), 
               Movie.IdType.iloc[i], Movie.IdPlatform.iloc[i], 
               Movie.ReleaseYear.iloc[i], duration, 
               str(Movie.TimeUnit.iloc[i]))
    cur.execute(query)
conn.commit()

In [23]:
Movie_Actor = pd.read_csv('./Tablas/Movie_Actor.csv')
Movie_Actor.head(3)

,IdMovie,IdActor
0,1,48068
1,1,49592
2,1,8756


In [24]:
for i in range(Movie_Actor.shape[0]):
    query ="""
    INSERT INTO Movie_Actor (IdMovie, IdActor)
    VALUES ({},{});
    """.format(Movie_Actor.IdMovie.iloc[i],Movie_Actor.IdActor.iloc[i])
    cur.execute(query)
conn.commit()

In [25]:
Movie_Genr = pd.read_csv('./Tablas/Movie_Genr.csv')
Movie_Genr.head(3)

,IdMovie,IdGenr
0,1,79
1,1,110
2,2,7


In [26]:
for i in range(Movie_Genr.shape[0]):
    query ="""
    INSERT INTO Movie_Genr (IdMovie, IdGenr)
    VALUES ({},{});
    """.format(Movie_Genr.IdMovie.iloc[i],Movie_Genr.IdGenr.iloc[i])
    cur.execute(query)
conn.commit()

In [27]:
conn.close()

## Hacer consultas a la base de datos

In [3]:
# Creamos una conección a la base de datos
conn = pymysql.connect(
    host='localhost',
    user='root',
    passwd='1234567890',
    database = 'streaming_platforms'
)

cur = conn.cursor()

### Consulta 1

In [9]:
year = 2018
platform = 'Hulu'
type_ = 'Movie'
time_unit = 'min'

query = """
SELECT m.Title, max(Duration) as Duration
FROM Movie m
JOIN Platform p ON m.IdPlatform = p.IdPlatform
JOIN Types t ON m.IdType = t.IdType
WHERE m.ReleaseYear = {} AND p.Platform = '{}' AND t.Type = '{}' AND m.TimeUnit = '{}';
""".format(year, platform, type_, time_unit)
cur.execute(query)

1

In [10]:
cur.fetchall()

(('This World Alone', 157),)

### Consulta 2

In [25]:
platform = 'NetFlix'

query = """
SELECT t.Type, count(*)
FROM Movie m
JOIN Platform p ON m.IdPlatform = p.IdPlatform
JOIN Types t ON m.IdType = t.IdType
WHERE p.Platform = '{}'
GROUP BY t.Type;
""".format(platform)

cur.execute(query)

2

In [26]:
cur.fetchall()

(('Movie', 6131), ('TV Show', 2676))

### Consulta 3

In [29]:
genr = 'Comedy'
query = """
SELECT a.Platform, max(a.Cantidad)
FROM (
    SELECT p.Platform, count(*) as Cantidad
    FROM Movie m
    JOIN Platform p ON m.IdPlatform = p.IdPlatform
    JOIN Movie_Genr mg ON m.IdMovie = mg.IdMovie
    JOIN Genr g ON mg.IdGenr = g.IdGenr
    WHERE g.Genr = '{}'
    GROUP BY p.Platform
    ) AS a;
""".format(genr)
cur.execute(query)

1

In [30]:
cur.fetchall()

(('Amazon', 2099),)

### Consulta 4

In [33]:
platform = 'Netflix'
year = 2018
query = """
SELECT a.Name, count(*) AS Cantidad
FROM Movie m
JOIN Platform p ON m.IdPlatform = p.IdPlatform
JOIN Movie_Actor ma ON m.IdMovie = ma.IdMovie
JOIN Actor a ON ma.IdActor = a.IdActor
WHERE p.Platform = '{}' AND m.ReleaseYear = {} AND a.Name != 'No Info'
GROUP BY a.Name
ORDER BY Cantidad DESC
LIMIT 1;
""".format(platform, year)
cur.execute(query)

1

In [34]:
cur.fetchall()

(('Andrea Libman', 8),)